In [1]:
%pip install ultralytics
%pip install powershell_kernel

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
!kaggle datasets download -d andrewmvd/car-plate-detection


  0%|          | 0.00/203M [00:00<?, ?B/s]
  0%|          | 1.00M/203M [00:00<02:42, 1.30MB/s]
  1%|          | 2.00M/203M [00:00<01:21, 2.58MB/s]
  2%|▏         | 4.00M/203M [00:01<00:41, 5.05MB/s]
  3%|▎         | 6.00M/203M [00:01<00:30, 6.87MB/s]
  4%|▍         | 8.00M/203M [00:01<00:24, 8.27MB/s]
  5%|▍         | 10.0M/203M [00:01<00:21, 9.23MB/s]
  6%|▌         | 12.0M/203M [00:01<00:20, 9.78MB/s]
  7%|▋         | 14.0M/203M [00:02<00:19, 10.2MB/s]
  8%|▊         | 16.0M/203M [00:02<00:18, 10.7MB/s]
  9%|▉         | 18.0M/203M [00:02<00:17, 11.0MB/s]
 10%|▉         | 20.0M/203M [00:02<00:17, 11.2MB/s]
 11%|█         | 22.0M/203M [00:02<00:16, 11.3MB/s]
 12%|█▏        | 24.0M/203M [00:02<00:16, 11.5MB/s]
 13%|█▎        | 26.0M/203M [00:03<00:16, 11.6MB/s]
 14%|█▍        | 28.0M/203M [00:03<00:16, 11.4MB/s]
 15%|█▍        | 30.0M/203M [00:03<00:16, 11.0MB/s]
 16%|█▌        | 32.0M/203M [00:03<00:16, 11.2MB/s]
 17%|█▋        | 34.0M/203M [00:03<00:15, 11.4MB/s]
 18%|█▊        | 36.

In [2]:
import xml.etree.ElementTree as ET
from tqdm import tqdm
from pathlib import Path
import shutil

def convert_label(path, lb_path):
  def convert_box(size, box):
    dw, dh = 1. / size[0], 1. / size[1]
    x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
    return x * dw, y * dh, w * dw, h * dh

  in_file = open(path)
  out_file = open(lb_path, 'w')
  tree = ET.parse(in_file)
  root = tree.getroot()
  size = root.find('size')
  w = int(size.find('width').text)
  h = int(size.find('height').text)

  names = ["licence"] # names list
  for obj in root.iter('object'):
    cls = obj.find('name').text
    if cls in names and int(obj.find('difficult').text) != 1:
      xmlbox = obj.find('bndbox')
      bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ('xmin', 'xmax', 'ymin', 'ymax')])
      cls_id = names.index(cls)  # class id
      out_file.write(" ".join([str(a) for a in (cls_id, *bb)]) + '\n')


In [3]:
path = Path("./yolo")
for dir in ("train" , "valid"):
  imgs_path = path / dir / 'images'
  lbs_path = path / dir / 'labels'
  imgs_path.mkdir(exist_ok=True, parents=True)
  lbs_path.mkdir(exist_ok=True, parents=True)

train_valid_ratio = 0.9
annotations = list(Path("./annotations/").glob("*.xml"))
for idx , an_path in enumerate(annotations):
  target_lb_path = path / 'train' / 'labels' if idx < train_valid_ratio * len(annotations) else path / 'valid' / 'labels'
  target_img_path = path / 'train' / 'images' if idx < train_valid_ratio * len(annotations) else path / 'valid' / 'images'
  lb_path = (target_lb_path / an_path.name).with_suffix('.txt')  # new label path
  shutil.copy( (Path("./images") / an_path.name).with_suffix('.png') , target_img_path)
  convert_label(an_path, lb_path)  # convert labels to YOLO format


In [4]:
yaml_file = """
path: ../yolo
train: train/images
val: valid/images

names:
 0: licence
"""
with open("car_plate.yaml" , 'w') as f:
  f.write(yaml_file)


In [1]:
import os
import shutil
import time
from ultralytics import YOLO
import torch

def train():
        train_path="./runs/detect/train"
        if os.path.exists(train_path):
            shutil.rmtree(train_path)
        model = YOLO("YOLOv8n.pt")
        print("Training.........")
        t1=time.time()
        model.train(data="./car_plate.yaml", epochs=150, imgsz=640)
        t2=time.time()
        print(f'Cost Time : {t2-t1}s')
        path=model.export()
        print(f'Output model : {path}')
if __name__=='__main__':
    if not torch.cuda.is_available():
        print("cuda is not available")
    else:
        print("cuda is available")
        train()


cuda is available
Training.........
Ultralytics YOLOv8.0.216 🚀 Python-3.10.13 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
engine\trainer: task=detect, mode=train, model=YOLOv8n.pt, data=./car_plate.yaml, epochs=150, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, bo

train: Scanning H:\Repos\WalkerGuard-EmbeddedSystem\yolov8\License_Plate\yolo\train\labels.cache... 390 images, 0 backgrounds, 0 corrupt: 100%|██████████| 390/390 [00:00<?, ?it/s]
val: Scanning H:\Repos\WalkerGuard-EmbeddedSystem\yolov8\License_Plate\yolo\valid\labels.cache... 43 images, 0 backgrounds, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/150      4.01G      1.504      2.907      1.304         10        640: 100%|██████████| 25/25 [00:05<00:00,  4.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.56it/s]

                   all         43         47    0.00524      0.745       0.66      0.336


...

    114/150      4.01G     0.8816     0.4914      1.007         12        640: 100%|██████████| 25/25 [00:04<00:00,  5.58it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.18it/s]

                   all         43         47      0.875      0.893      0.905      0.487
Stopping training early as no improvement observed in last 50 epochs. Best results observed at epoch 64, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



114 epochs completed in 0.181 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.0.216 🚀 Python-3.10.13 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.89it/s]


                   all         43         47      0.952      0.849      0.944      0.516
Speed: 0.8ms preprocess, 2.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train
Cost Time : 698.6870710849762s
Ultralytics YOLOv8.0.216 🚀 Python-3.10.13 torch-2.0.1+cu118 CPU (Intel Core(TM) i7-9700KF 3.60GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

TorchScript: starting export with torch 2.0.1+cu118...
TorchScript: export success ✅ 1.9s, saved as 'runs\detect\train\weights\best.torchscript' (11.9 MB)

Export complete (3.8s)
Results saved to H:\Repos\WalkerGuard-EmbeddedSystem\yolov8\License_Plate\runs\detect\train\weights
Predict:         yolo predict task=detect model=runs\detect\train\weights\best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs\detect\train

In [4]:

!yolo export model="./runs/detect/train/weights/best.pt" format=tflite int8=true

Ultralytics YOLOv8.0.216 🚀 Python-3.10.13 torch-2.0.1+cu118 CPU (Intel Core(TM) i7-9700KF 3.60GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

TensorFlow SavedModel: starting export with tensorflow 2.10.0...

ONNX: starting export with onnx 1.15.0 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success ✅ 2.2s, saved as 'runs\detect\train\weights\best.onnx' (11.6 MB)
TensorFlow SavedModel: running 'onnx2tf -i "runs\detect\train\weights\best.onnx" -o "runs\detect\train\weights\best_saved_model" -nuo --verbosity info -oiqt -qt per-tensor'

Automatic generation of each OP name started ===============

fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8
Traceback (most recent call last):
  File "C:\Users\henry\.conda\envs\ML\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\henry\.conda\envs\ML\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\henry\.conda\envs\ML\Scripts\onnx2tf.exe\__main__.py", line 7, in <module>
  File "C:\Users\henry\.conda\envs\ML\lib\site-packages\onnx2tf\onnx2tf.py", line 2291, in main
    model = convert(
  File "C:\Users\henry\.conda\envs\ML\lib\site-packages\onnx2tf\onnx2tf.py", line 1525, in convert
    tflite_model = converter.convert()
  File "C:\Users\henry\.conda\envs\ML\lib\site-packages\tensorflow\lite\python\lite.py", line 930, in wrapper
    return self._convert_and_export_metrics(convert_func, *args, **kwargs)
  F

Ultralytics YOLOv8.0.216 🚀 Python-3.10.13 torch-2.0.1+cu118 CPU (Intel Core(TM) i7-9700KF 3.60GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

TensorFlow SavedModel: starting export with tensorflow 2.10.0...

ONNX: starting export with onnx 1.15.0 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success ✅ 2.5s, saved as 'runs\detect\train\weights\best.onnx' (11.6 MB)
TensorFlow SavedModel: running 'onnx2tf -i "runs\detect\train\weights\best.onnx" -o "runs\detect\train\weights\best_saved_model" -nuo --verbosity info -oiqt -qt per-tensor'

Automatic generation of each OP name started ===============

fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8
2023-11-23 23:29:11.839860: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
